# 기본 로드

##### 라이브러리 로드

In [1]:
# 기초 전처리
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm
from math import log as ln
%matplotlib inline

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

##### 필요 함수 정의

In [2]:
def 로드_및_전처리(data):
    
    ### 데이터를 로드하여 전처리하는 함수 ###
    df = pd.read_csv(data, encoding='cp949')
    df = df[df['상권_구분_코드']=='A']
    df['상권_코드'] = df['상권_코드'].astype('str')
    df['cat서비스_업종_코드_명'] = df['서비스_업종_코드_명'].apply(categorization)
    
    if '_2020.csv' in data:
        # 2020년 3분기, 2020년 4분기, 2021년 1분기, 2021년 2분기 데이터만 추출하기 위한 조건 생성
        기준년_2020_3분기= ((df['기준_년_코드']==2020) & (df['기준_분기_코드']==3)) 
        기준년_2020_4분기= ((df['기준_년_코드']==2020) & (df['기준_분기_코드']==4)) 
        # 전체 테이블에 boolean 인덱싱
        df = df[ 기준년_2020_3분기 | 기준년_2020_4분기 ]
        return df
    else:
        기준년_2021_1분기= ((df['기준_년_코드']==2021) & (df['기준_분기_코드']==1)) 
        기준년_2021_2분기= ((df['기준_년_코드']==2021) & (df['기준_분기_코드']==2))
        
        df = df[ 기준년_2021_1분기 | 기준년_2021_2분기 ]
        return df

def categorization(x):
    if x == '청과상' or x == '수산물판매' or x == '육류판매' or x == '미곡판매' or x == '반찬가게' or \
       x == '치과의원' or x == '의료기기' or x == '의약품' or x == '한의원' or x == '일반의원' or \
       x == '편의점' or x == '슈퍼마켓' or x == '세탁소' or x == '제과점':
        return '생활 필수'
    
    elif x == '미용재료' or x == '화장품' or x == '피부관리실' or x == '네일숍' or x == '미용실' or \
         x == '한복점' or x ==  '섬유제품' or x ==  '유아의류' or x == '안경' or x == '신발' or \
         x == '가방' or x == '의류임대' or x == '일반의류':
        return '미용/의류'
    
    elif x == '노래방' or x == '전자게임장' or x == 'PC방' or x == '골프연습장' or x == '당구장' or \
         x == 'DVD방' or x == '기타오락장' or x == '볼링장' or x == '스포츠클럽' or x == '스포츠 강습' or \
         x == '게스트하우스' or x == '여행사' or x == '서적':
        return '여가'
    
    elif x == '분식전문점' or x == '커피-음료' or x == '호프-간이주점' or x == '일식음식점' or \
         x == '중식음식점' or x == '한식음식점' or x == '양식음식점' or x == '패스트푸드점' or x == '치킨전문점':
        return '외식'
    
    elif x == '세무사사무소' or x == '법무사사무소' or x == '기타법무서비스' or x == '변리사사무소' or x == '회계사사무소' or \
         x == '변호사사무소' or x == '부동산중개업' or x == '독서실' or x == '문구' or x == '컴퓨터학원' or x == '고시원' or \
         x == '예술학원' or x == '외국어학원' or x == '일반교습학원' or x == '가전제품' or x == '핸드폰' or x == '통신기기수리' or \
         x == '컴퓨터및주변장치판매' or x == '가전제품수리' or x == '자동차미용' or x == '자동차수리' or x == '중고차판매' or \
         x == '자전거 및 기타운송장비' or x == '모터사이클수리'  or x == '주류도매':
        return '편의시설'
    
    else:
        return '기타'

In [3]:
def 상권코드_str(df):
    df.reset_index(inplace=True)
    df['상권_코드'] = df['상권_코드'].astype(str)
    return df

In [4]:
df_base = pd.read_csv('base_df.csv')
df_base['상권_코드'] = df_base['상권_코드'].astype(str)
df_base.head(3)

,상권_코드
0,1000001
1,1000002
2,1000003


# Summary - 상권의 고객은 누구이며 무엇때문에 오는가?

##### 단순지표

1. 성별 매출 금액
2. 소득 수준

##### 복합지표
1. 상권의 다양성 => 하나의 상권에 얼마나 다양한 종류의 점포가 섞여 있는가?
    - Shannon index
2. 고객 유입성
    - 인근 거주자 
        - 생활 인구수/총매출
    - 외부 거주자
        - 직장 인구수/총매출

# Summary - 출처 데이터
- 1년_카테고리_테이블.csv: 서울시 우리마을가게 상권분석서비스(상권-추정매출)을 전처리한 테이블
    - 사용처: 상권의 성별 매출 금액
- 서울시 우리마을가게 상권분석서비스(상권배후지-소득소비)
    - 사용처: 소득 수준
-  서울시 우리마을가게 상권분석서비스(상권-점포)
    - 사용처: 상권의 다양성

# 단순지표

##### 단순지표

1. 업종별 매출 금액
2. 성별 매출 금액
3. 소득 수준

## 요일별 매출 금액

In [5]:
df = pd.read_csv('1년_카테고리_테이블.csv')
df.head(3)

,상권_코드,서비스_업종_코드_명,분기당_매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,cat서비스_업종_코드_명
0,1000127,당구장,424034.0,424034.0,0.0,0,0.0,0,324463,99571,0,0,0,0,0.0,0.0,99571.0,324463,424034.0,0.0,0,0,424034,0.0,0,0,여가
1,1000166,네일숍,252242.0,252242.0,0.0,0,0.0,0,252242,0,0,0,0,0,0.0,252242.0,0.0,0,252242.0,0.0,0,0,0,252242.0,0,0,미용/의류
2,1000184,외국어학원,21642943.0,21642943.0,0.0,5166677,4174652.0,8955575,0,3346039,0,0,0,0,1328574.0,6003187.0,14311182.0,0,12441337.0,8217477.0,0,0,0,13277845.0,7380969,0,편의시설


In [6]:
# 쓸만한 테이블로 압축
week_table = df[['상권_코드', '월요일_매출_금액','화요일_매출_금액',
                 '수요일_매출_금액','목요일_매출_금액','금요일_매출_금액',
                 '토요일_매출_금액','일요일_매출_금액']]
week_table.head(3)

,상권_코드,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액
0,1000127,0,0.0,0,324463,99571,0,0
1,1000166,0,0.0,0,252242,0,0,0
2,1000184,5166677,4174652.0,8955575,0,3346039,0,0


In [7]:
##### 상권코드별 요일별 매출 비율
grouped =  week_table.groupby(['상권_코드']).sum()

# loop를 돌아 총 매출 비율 테이블 수정
for index in range(grouped.shape[0]):
    grouped_sum = grouped.iloc[index].sum() # 열별 총 요일 매출 금액
    grouped.iloc[index,0] = grouped.iloc[index,0]/grouped_sum
    grouped.iloc[index,1] = grouped.iloc[index,1]/grouped_sum
    grouped.iloc[index,2] = grouped.iloc[index,2]/grouped_sum
    grouped.iloc[index,3] = grouped.iloc[index,3]/grouped_sum
    grouped.iloc[index,4] = grouped.iloc[index,4]/grouped_sum
    grouped.iloc[index,5] = grouped.iloc[index,5]/grouped_sum
    grouped.iloc[index,6] = grouped.iloc[index,6]/grouped_sum
    
grouped = 상권코드_str(grouped)
grouped.head()

,상권_코드,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액
0,1000001,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823
1,1000002,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143
2,1000003,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239
3,1000004,0.156436,0.160766,0.167509,0.139482,0.157609,0.123508,0.094691
4,1000005,0.080943,0.118395,0.140430,0.113667,0.141985,0.217003,0.187577


In [9]:
df_pre = df_base['상권_코드']
df_pre = pd.merge(df_pre, grouped, on='상권_코드', how='left')
df_pre.head(3)

,상권_코드,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액
0,1000001,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823
1,1000002,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143
2,1000003,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239


## 소득 수준

In [10]:
income = pd.read_csv('./상권_소득소비/상권_소득.csv')
income = income[['상권_코드','소득_금액']]
income['상권_코드'] = income['상권_코드'].astype(str)
income = pd.merge(df_base, income, on=['상권_코드'], how='left')
income.head(2)

,상권_코드,소득_금액
0,1000001,56459184
1,1000002,52528944


In [11]:
income['소득_비율'] = income['소득_금액']/sum(income['소득_금액'])
income.drop('소득_금액',axis=1, inplace=True)
income.head(3)

,상권_코드,소득_비율
0,1000001,0.001132
1,1000002,0.001053
2,1000003,0.000933


In [12]:
df_pre = pd.merge(df_pre, income, on='상권_코드', how='left')
df_pre.head(3)

,상권_코드,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,소득_비율
0,1000001,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.001132
1,1000002,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.001053
2,1000003,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.000933


# 복합지표

## 상권의 다양성 
- 하나의 상권에 얼마나 다양한 종류의 점포가 섞여 있는가?
- Shannon index를 활용
    $$H = - \sum^S_{i=0}p_i\ln{p_i}$$
S: 클래스의 수  
$p_i$: 클래스 i에 대한 데이터의 수
- Shannon index(Shannon-Weaver)란?
    - 한 집단 내의 데이터의 class가 얼마나 다양한지를 나타내는 지표
    - 클래스의 갯수 뿐만이 아니라 각 클래스에 속한 자료의 분포까지 고려하여 수치를 계산  
    
[출처](https://medium.com/@Aaron__Kim/diversity-index-%EB%8B%A4%EC%96%91%EC%84%B1-%EB%B6%84%EC%84%9D-shannons-diversity-information-gain-entropy-3e0dbd268e9e)

In [13]:
# 기본 로드
diversity = pd.read_csv('./상권_점포/서울시 우리마을가게 상권분석서비스(상권-점포).csv', encoding = 'cp949')
print(diversity.shape)

# 골목상권만 추출
diversity = diversity[diversity['상권_구분_코드']=='A']

# 2020년과 2021년 데이터만 추출
diversity = diversity[(diversity['기준_년_코드']==2021) | (diversity['기준_년_코드']==2020)]

# 2020년 3분기, 2020년 4분기, 2021년 1분기, 2021년 2분기 데이터만 추출하기 위한 조건 생성
기준년_2020_3분기= ((diversity['기준_년_코드']==2020) & (diversity['기준_분기_코드']==3)) 
기준년_2020_4분기= ((diversity['기준_년_코드']==2020) & (diversity['기준_분기_코드']==4)) 
기준년_2021_1분기= ((diversity['기준_년_코드']==2021) & (diversity['기준_분기_코드']==1)) 
기준년_2021_2분기= ((diversity['기준_년_코드']==2021) & (diversity['기준_분기_코드']==2))

# 전체 테이블에 boolean 인덱싱
diversity = diversity[ 기준년_2020_3분기 | 기준년_2020_4분기 | 기준년_2021_1분기 | 기준년_2021_2분기 ]

# 점포수가 없는데 왜 테이블에 있지?
diversity = diversity[diversity['점포_수'] != 0]

# 상권_코드 str처리
diversity['상권_코드'] = diversity['상권_코드'].astype(str)
diversity.head(3)

(555241, 15)


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
29738,2021,2,A,골목상권,1001010,풍성로37가길,CS300043,전자상거래업,5,5,0,0,0,0,0
29740,2021,2,A,골목상권,1001010,풍성로37가길,CS300036,조명용품,3,3,0,0,0,0,0
29741,2021,2,A,골목상권,1001010,풍성로37가길,CS300035,인테리어,1,1,0,0,0,0,0


In [14]:
# 상권코드를 오름차순으로 정렬하여 list화 시킴
총_상권_코드 = sorted(list(map(int,diversity['상권_코드'].unique())))

# 다양성 지수를 추가하기 위한 데이터프레임
total = pd.DataFrame({'상권_코드': 총_상권_코드,
                      '다양성_지수': 총_상권_코드})

# 데이터를 추가할 때의 해당 열 번호
idx_num = 0

for index in 총_상권_코드:
    ##### Shannon-Weaver를 구하기 위한 loop #####
    # 정렬된 상권코드와 일치한 점포 테이블을 불러옴
    store_code = diversity[diversity['상권_코드']== str(index)]
    
    # 해당 상권_코드별로 유사_업종_점포_수별 합계(eg.음식점=30 / 서점=10)를 구함
    grouped_store = store_code.groupby(['서비스_업종_코드_명'])['유사_업종_점포_수'].sum()
    
    # 해당 상권_코드별 총 유사_업종_점포_수의 합계(eg. 총 점포수=40)
    N = sum(grouped_store)
    
    # 상권코드별 총 점포 Shannon-Weaver 지수를 위한 초기화 변수
    result = 0
    for store in grouped_store:
        cnt = 0  # 해당 점포(e.g 음식점)의 Shannon-Weaver 지수를 구하기 위한 초기화 변수
        cnt = float(store)/N * ln(float(store)/N) # Shannon-Weaver 지수 식
        result += cnt # 상권코드별 총 점포 Shannon-Weaver 지수를 계산
    total.iloc[idx_num,1]= -result # 총 Shannon-Weaver 지수를 테이블에 추가
    idx_num += 1 # 테이블 열 인덱스를 하나씩 추가한다.
    
total.head()

,상권_코드,다양성_지수
0,1000001,3.431027
1,1000002,3.090867
2,1000003,3.492629
3,1000004,2.480483
4,1000005,3.274153


In [15]:
total['상권_코드'] = total['상권_코드'].astype(str)
total = pd.merge(df_base, total, on='상권_코드', how='left')
total.head()

,상권_코드,다양성_지수
0,1000001,3.431027
1,1000002,3.090867
2,1000003,3.492629
3,1000004,2.480483
4,1000005,3.274153


In [16]:
df_pre = pd.merge(df_pre, total, on='상권_코드', how='left')
df_pre.head(3)

,상권_코드,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,소득_비율,다양성_지수
0,1000001,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.001132,3.431027
1,1000002,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.001053,3.090867
2,1000003,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.000933,3.492629


## 고객 유입성 - 인근 거주자

In [17]:
df = pd.read_csv('./상권_인구/상권_생활인구.csv')
print(df.shape)
df.head(3)

(1010, 527)


,상권_코드,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,연령대_20_생활인구_수,연령대_30_생활인구_수,연령대_40_생활인구_수,연령대_50_생활인구_수,연령대_60_이상_생활인구_수,시간대_1_생활인구_수,시간대_2_생활인구_수,시간대_3_생활인구_수,시간대_4_생활인구_수,시간대_5_생활인구_수,시간대_6_생활인구_수,월요일_생활인구_수,화요일_생활인구_수,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수,남성연령대_10_월요일시간대_1_생활인구_수,남성연령대_10_월요일시간대_2_생활인구_수,남성연령대_10_월요일시간대_3_생활인구_수,남성연령대_10_월요일시간대_4_생활인구_수,남성연령대_10_월요일시간대_5_생활인구_수,남성연령대_10_월요일시간대_6_생활인구_수,남성연령대_10_화요일시간대_1_생활인구_수,남성연령대_10_화요일시간대_2_생활인구_수,남성연령대_10_화요일시간대_3_생활인구_수,남성연령대_10_화요일시간대_4_생활인구_수,남성연령대_10_화요일시간대_5_생활인구_수,남성연령대_10_화요일시간대_6_생활인구_수,남성연령대_10_수요일시간대_1_생활인구_수,남성연령대_10_수요일시간대_2_생활인구_수,남성연령대_10_수요일시간대_3_생활인구_수,남성연령대_10_수요일시간대_4_생활인구_수,남성연령대_10_수요일시간대_5_생활인구_수,남성연령대_10_수요일시간대_6_생활인구_수,남성연령대_10_목요일시간대_1_생활인구_수,남성연령대_10_목요일시간대_2_생활인구_수,남성연령대_10_목요일시간대_3_생활인구_수,남성연령대_10_목요일시간대_4_생활인구_수,남성연령대_10_목요일시간대_5_생활인구_수,남성연령대_10_목요일시간대_6_생활인구_수,남성연령대_10_금요일시간대_1_생활인구_수,남성연령대_10_금요일시간대_2_생활인구_수,남성연령대_10_금요일시간대_3_생활인구_수,남성연령대_10_금요일시간대_4_생활인구_수,남성연령대_10_금요일시간대_5_생활인구_수,남성연령대_10_금요일시간대_6_생활인구_수,남성연령대_10_토요일시간대_1_생활인구_수,남성연령대_10_토요일시간대_2_생활인구_수,남성연령대_10_토요일시간대_3_생활인구_수,남성연령대_10_토요일시간대_4_생활인구_수,남성연령대_10_토요일시간대_5_생활인구_수,남성연령대_10_토요일시간대_6_생활인구_수,남성연령대_10_일요일시간대_1_생활인구_수,남성연령대_10_일요일시간대_2_생활인구_수,남성연령대_10_일요일시간대_3_생활인구_수,남성연령대_10_일요일시간대_4_생활인구_수,남성연령대_10_일요일시간대_5_생활인구_수,남성연령대_10_일요일시간대_6_생활인구_수,남성연령대_20_월요일시간대_1_생활인구_수,남성연령대_20_월요일시간대_2_생활인구_수,남성연령대_20_월요일시간대_3_생활인구_수,남성연령대_20_월요일시간대_4_생활인구_수,남성연령대_20_월요일시간대_5_생활인구_수,남성연령대_20_월요일시간대_6_생활인구_수,남성연령대_20_화요일시간대_1_생활인구_수,남성연령대_20_화요일시간대_2_생활인구_수,남성연령대_20_화요일시간대_3_생활인구_수,남성연령대_20_화요일시간대_4_생활인구_수,남성연령대_20_화요일시간대_5_생활인구_수,남성연령대_20_화요일시간대_6_생활인구_수,남성연령대_20_수요일시간대_1_생활인구_수,남성연령대_20_수요일시간대_2_생활인구_수,남성연령대_20_수요일시간대_3_생활인구_수,남성연령대_20_수요일시간대_4_생활인구_수,남성연령대_20_수요일시간대_5_생활인구_수,남성연령대_20_수요일시간대_6_생활인구_수,남성연령대_20_목요일시간대_1_생활인구_수,남성연령대_20_목요일시간대_2_생활인구_수,남성연령대_20_목요일시간대_3_생활인구_수,남성연령대_20_목요일시간대_4_생활인구_수,남성연령대_20_목요일시간대_5_생활인구_수,남성연령대_20_목요일시간대_6_생활인구_수,남성연령대_20_금요일시간대_1_생활인구_수,남성연령대_20_금요일시간대_2_생활인구_수,남성연령대_20_금요일시간대_3_생활인구_수,남성연령대_20_금요일시간대_4_생활인구_수,남성연령대_20_금요일시간대_5_생활인구_수,남성연령대_20_금요일시간대_6_생활인구_수,남성연령대_20_토요일시간대_1_생활인구_수,남성연령대_20_토요일시간대_2_생활인구_수,남성연령대_20_토요일시간대_3_생활인구_수,남성연령대_20_토요일시간대_4_생활인구_수,남성연령대_20_토요일시간대_5_생활인구_수,남성연령대_20_토요일시간대_6_생활인구_수,남성연령대_20_일요일시간대_1_생활인구_수,남성연령대_20_일요일시간대_2_생활인구_수,남성연령대_20_일요일시간대_3_생활인구_수,남성연령대_20_일요일시간대_4_생활인구_수,남성연령대_20_일요일시간대_5_생활인구_수,남성연령대_20_일요일시간대_6_생활인구_수,남성연령대_30_월요일시간대_1_생활인구_수,남성연령대_30_월요일시간대_2_생활인구_수,남성연령대_30_월요일시간대_3_생활인구_수,남성연령대_30_월요일시간대_4_생활인구_수,남성연령대_30_월요일시간대_5_생활인구_수,남성연령대_30_월요일시간대_6_생활인구_수,남성연령대_30_화요일시간대_1_생활인구_수,남성연령대_30_화요일시간대_2_생활인구_수,남성연령대_30_화요일시간대_3_생활인구_수,남성연령대_30_화요일시간대_4_생활인구_수,남성연령대_30_화요일시간대_5_생활인구_수,남성연령대_30_화요일시간대_6_생활인구_수,남성연령대_30_수요일시간대_1_생활인구_수,남성연령대_30_수요일시간대_2_생활인구_수,남성연령대_30_수요일시간대_3_생활인구_수,남성연령대_30_수요일시간대_4_생활인구_수,남성연령대_30_수요일시간대_5_생활인구_수,남성연령대_30_수요일시간대_6_생활인구_수,남성연령대_30_목요일시간대_1_생활인구_수,남성연령대_30_목요일시간대_2_생활인구_수,남성연령대_30_목요일시간대_3_생활인구_수,남성연령대_30_목요일시간대_4_생활인구_수,남성연령대_30_목요일시간대_5_생활인구_수,남성연령대_30_목요일시간대_6_생활인구_수,남성연령대_30_금요일시간대_1_생활인구_수,남성연령대_30_금요일시간대_2_생활인구_수,남성연령대_30_금요일시간대_3_생활인구_수,남성연령대_30_금요일시간대_4_생활인구_수,남성연령대_30_금요일시간대_5_생활인구_수,남성연령대_30_금요일시간대_6_생활인구_수,남성연령대_30_토요일시간대_1_생활인구_수,남성연령대_30_토요일시간대_2_생활인구_수,남성연령대_30_토요일시간대_3_생활인구_수,남성연령대_30_토요일시간대_4_생활인구_수,남성연령대_30_토요일시간대_5_생활인구_수,남성연령대_30_토요일시간대_6_생활인구_수,남성연령대_30_일요일시간대_1_생활인구_수,남성연령대_30_일요일시간대_2_생활인구_수,남성연령대_30_일요일시간대_3_생활인구_수,남성연령대_30_일요일시간대_4_생활인구_수,남성연령대_30_일요일시간대_5_생활인구_수,남성연령대_30_일요일시간대_6_생활인구_수,남성연령대_40_월요일시간대_1_생활인구_수,남성연령대_40_월요일시간대_2_생활인구_수,남성연령대_40_월요일시간대_3_생활인구_수,남성연령대_40_월요일시간대_4_생활인구_수,남성연령대_40_월요일시간대_5_생활인구_수,남성연령대_40_월요일시간대_6_생활인구_수,남성연령대_40_화요일시간대_1_생활인구_수,남성연령대_40_화요일시간대_2_생활인구_수,남성연령대_40_화요일시간대_3_생활인구_수,남성연령대_40_화요일시간대_4_생활인구_수,남성연령대_40_화요일시간대_5_생활인구_수,남성연령대_40_화요일시간대_6_생활인구_수,남성연령대_40_수요일시간대_1_생활인구_수,남성연령대_40_수요일시간대_2_생활인구_수,남성연령대_40_수요일시간대_3_생활인구_수,남성연령대_40_수요일시간대_4_생활인구_수,남성연령대_40_수요일시간대_5_생활인구_수,남성연령대_40_수요일시간대_6_생활인구_수,남성연령대_40_목요일시간대_1_생활인구_수,남성연령대_40_목요일시간대_2_생활인구_수,남성연령대_40_목요일시간대_3_생활인구_수,남성연령대_40_목요일시간대_4_생활인구_수,남성연령대_40_목요일시간대_5_생활인구

In [18]:
total_income = pd.read_csv('1년_카테고리_테이블.csv')

# 필요있는 컬럼만 로드
total_income = total_income[['상권_코드','분기당_매출_금액']]

# 상권코드별 집계
total_income = total_income.groupby(['상권_코드']).sum()

# 테이블 재구조화
total_income.reset_index(inplace=True)
total_income['상권_코드'] = total_income['상권_코드'].astype(str)

# base 테이블과 merge
total_income = pd.merge(df_base, total_income, on='상권_코드', how='left')
total_income.head(3)

,상권_코드,분기당_매출_금액
0,1000001,1.512657e+10
1,1000002,1.599466e+10
2,1000003,2.874329e+10


In [19]:
inner_customer = pd.DataFrame()
inner_customer['상권_코드'] = df['상권_코드']
inner_customer['내부_고객_유입률'] = df['총_생활인구_수']/total_income['분기당_매출_금액']
inner_customer['상권_코드'] = inner_customer['상권_코드'].astype(str)
inner_customer.head()

,상권_코드,내부_고객_유입률
0,1000001,0.000286
1,1000002,0.000911
2,1000003,0.000241
3,1000004,0.001181
4,1000005,0.000176


In [20]:
df_pre = pd.merge(df_pre, inner_customer, on='상권_코드', how='left')
df_pre.head(3)

,상권_코드,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,소득_비율,다양성_지수,내부_고객_유입률
0,1000001,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.001132,3.431027,0.000286
1,1000002,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.001053,3.090867,0.000911
2,1000003,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.000933,3.492629,0.000241


## 고객 유입성 - 외부 거주자

In [21]:
df = pd.read_csv('./상권_직장인구/상권_직장인구.csv')

# 필요한 컬럼만 로드
df = df[['상권_코드', '총_직장_인구_수']]

# 상권코드별로 집계
df = df.groupby(['상권_코드']).sum() 

# 테이블 재구조화
df.reset_index(inplace=True)
df['상권_코드'] = df['상권_코드'].astype(str)

df.head(3)

,상권_코드,총_직장_인구_수
0,1000001,54800
1,1000002,47092
2,1000003,158024


In [22]:
total_income.head(3)

,상권_코드,분기당_매출_금액
0,1000001,1.512657e+10
1,1000002,1.599466e+10
2,1000003,2.874329e+10


In [23]:
outer_customer = pd.DataFrame()
outer_customer['상권_코드'] = df['상권_코드']
outer_customer['외부_고객_유입률'] = df['총_직장_인구_수']/total_income['분기당_매출_금액']
outer_customer.head()

,상권_코드,외부_고객_유입률
0,1000001,3.622764e-06
1,1000002,2.944232e-06
2,1000003,5.497769e-06
3,1000004,6.327144e-07
4,1000005,1.629939e-06


In [24]:
df_pre = pd.merge(df_pre, outer_customer, on='상권_코드', how='left')
df_pre.head(3)

,상권_코드,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,소득_비율,다양성_지수,내부_고객_유입률,외부_고객_유입률
0,1000001,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.001132,3.431027,0.000286,0.000004
1,1000002,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.001053,3.090867,0.000911,0.000003
2,1000003,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.000933,3.492629,0.000241,0.000005
